In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [17]:
batch_size = 64
block_size = 128
max_iters = 1000
learning_rate = 3e-4
eval_iters = 100
# eval_interval = 500
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

In [18]:
with open('../data/vocab.txt','r',encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [19]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda x: [string_to_int[c] for c in x]
decode = lambda l: ''.join(int_to_string[x] for x in l)

data = torch.tensor(encode(text), dtype=torch.long)


In [20]:
# n = int(0.8 * len(data))
# train_data = data[:n]
# val_data = data[n:]
# 
# def get_batch(split):
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size,(batch_size,))
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x,y = x.to(device), y.to(device)
#     return x,y

In [21]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "../data/output_train.txt" if split == 'train' else "../data/output_val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, file_size - block_size * batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [22]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [23]:
class Head(nn.Module):
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0,float('-inf'))
        wei = F.softmax(wei,dim=-1)
        wei = self.dropout(wei)
        
        v = self.value(x)
        out = wei @ v
        return out

In [24]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads,n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads],dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [25]:
class FeedForward(nn.Module):
    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features=n_embd, out_features=4*n_embd),
            nn.ReLU(),
            nn.Linear(in_features=4*n_embd,out_features=n_embd),
            nn.Dropout(dropout),
        )
        
    def forward(self,x):
        return self.net(x)

In [26]:
class Block(nn.Module):
    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self,x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(y+x)
        return  x
        

In [27]:
class GPT(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self,module):
        if isinstance(module,nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module,nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0, std=0.02)         
        
        
    def forward(self,index,targets = None):
        B,T = index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
            
        return logits,loss
    
    def generate(self,index,max_new_token):
        
        for _ in range(max_new_token):
            logits,loss = self.forward(index)
            print(logits)
            logits = logits[:,-1,:]
            print(logits)
            probs = F.softmax(logits,dim=-1)
            index_next = torch.multinomial(probs,num_samples=1)
            index = torch.cat((index,index_next),dim=1)
        
        return index

In [29]:
model = GPT(vocab_size=vocab_size)
# print("Loading model parameters...")
# with open('../models/model-01.pkl','rb') as f:
#     model = pickle.load(f)
# print("loaded")
m = model.to(device)

In [30]:
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step : {iter}, Train loss : {losses['train']}, Validation loss : {losses['val']}")
        
    xb,yb = get_batch('train')
    
    logits,loss = m.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

Step : 0, Train loss : 8.84799575805664, Validation loss : 8.8462553024292
Step : 100, Train loss : 2.377779245376587, Validation loss : 2.352424383163452
Step : 200, Train loss : 2.271552801132202, Validation loss : 2.2983341217041016
Step : 300, Train loss : 2.3528852462768555, Validation loss : 2.3014888763427734
Step : 400, Train loss : 2.1695237159729004, Validation loss : 2.1124815940856934
Step : 500, Train loss : 2.044499635696411, Validation loss : 2.062140464782715
Step : 600, Train loss : 2.0709173679351807, Validation loss : 1.9742611646652222
Step : 700, Train loss : 1.9215087890625, Validation loss : 1.9437257051467896
Step : 800, Train loss : 1.8039458990097046, Validation loss : 1.8540457487106323
Step : 900, Train loss : 1.8590102195739746, Validation loss : 1.7575350999832153
1.411489725112915


In [31]:
with open('../models/model-03.pkl','wb') as f:
    pickle.dump(model, f)